# Depth-band plots

Loads depth-frequency-time data and then plots integrated depth-band power for various circumstances.

## Imports
Necessary modules for analysis.

In [1]:
# import modules

import xarray as xr
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.dates as pldates
import numpy as np
for i in range(2):
    %matplotlib notebook

In [13]:
# import data

adcp = 'Axis75'     # Slope(2013,2014,2017,2018), Axis75(2013,2014), Axis55(2017,2018)
year = 2013
ds_in = xr.open_dataset(f'../Data/data/spectra/spectra_{adcp}_{year}_0.nc')

n_seg = ds_in.n_seg
if n_seg > 1:
    ds = [ds_in]
    for i in range(n_seg):
        if i > 0:
            ds_temp = xr.open_dataset(f'../Data/data/spectra/spectra_{adcp}_{year}_{i}.nc')
            ds.append(ds_temp)
elif n_seg == 1:
    ds = [ds_in]

#print(ds)

In [14]:
# extract plotting variables

t_stamp = int(ds[0].t_stamp)
start_date = dt.datetime(t_stamp,1,1)                                   # input start date in YYYY,MM,DD
end_date = dt.datetime(t_stamp+1,1,1)                                     # input end date in YYYY,MM,DD
depth = ds[0].depth.values
d = len(depth)

In [15]:
# select frequency band

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx      # returns index of nearest value

f_array = np.asarray(ds[0].specf_s)    # input array to process 

f_name = 'Diurnal'          # Diurnal, Inertial, Semi-diurnal, Continuum
if f_name == 'Diurnal':
    f_freq = 1.15e-5         # desired frequency constituent
    f_idx = find_nearest(f_array, f_freq)                 # index of desired frequency
    up_f_idx = f_idx + 1
    low_f_idx = f_idx 
elif f_name == 'Inertial':
    f_freq = 1.73e-5
    f_idx = find_nearest(f_array, f_freq)                 # index of desired frequency
    up_f_idx = f_idx + 1
    low_f_idx = f_idx - 1 

print("Desired frequency band:",f_name,'at', f_freq, "Hz")
print("---")
print("Index at upper freq cutoff:", up_f_idx)
print("Value at upper freq cutoff:", f_array[up_f_idx], "Hz" )
print('---')
print("Index at lower freq cutoff:", low_f_idx)
print("Value at lower freq cutoff:", f_array[low_f_idx], "Hz" )

f_band = f_array[low_f_idx:up_f_idx+1]              # remove unwanted frequencies
fup_stamp = f_band[-1]                              # freq stamps for use in output
flow_stamp = f_band[0]

print('---')
print("Length of new freq array: ", len(f_band))   # new freq interval
print(f'Upper limit at',fup_stamp,'Hz')            # freq upper limit
print(f'Lower limit at',flow_stamp,'Hz')           # freq lower limit

Desired frequency band: Diurnal at 1.15e-05 Hz
---
Index at upper freq cutoff: 6
Value at upper freq cutoff: 1.3007812500000002e-05 Hz
---
Index at lower freq cutoff: 5
Value at lower freq cutoff: 1.083984375e-05 Hz
---
Length of new freq array:  2
Upper limit at 1.3007812500000002e-05 Hz
Lower limit at 1.083984375e-05 Hz


## Depth-band PSD spectra

In [16]:
# integrate over band for each time step

u_power, v_power = [],[]

for i in range(n_seg):
    t = len(ds[i].spect_s)
    u_depth_power_temp,v_depth_power_temp = [],[]
    for j in range(d):
        u_time_power_temp,v_time_power_temp = [],[]
        for k in range(t):
            u_PSD_temp = ds[i].Sxxu_d[j,low_f_idx:up_f_idx+1,k]
            u_power_temp = np.trapz(y=u_PSD_temp,x=f_band)
            u_time_power_temp.append(u_power_temp)
            v_PSD_temp = ds[i].Sxxv_d[j,low_f_idx:up_f_idx+1,k]
            v_power_temp = np.trapz(y=v_PSD_temp,x=f_band)
            v_time_power_temp.append(v_power_temp)
        u_depth_power_temp.append(u_time_power_temp)
        v_depth_power_temp.append(v_time_power_temp)
    u_np = np.asarray(u_depth_power_temp)
    v_np = np.asarray(v_depth_power_temp)
    u_power.append(u_np)            # n_seg, [depth, time]
    v_power.append(v_np)            # n_seg, [depth, time]

In [17]:
# WKB scale at each depth

u_WKB,v_WKB = [],[]
for i in range(n_seg):
    u_WKB_temp,v_WKB_temp = [],[]
    for j in range(d):
        u_WKB_temp.append(u_power[i][j] / ds[0].WKB_scale[j].values)          # list[depth][frequency]
        v_WKB_temp.append(v_power[i][j] / ds[0].WKB_scale[j].values)          # list[depth][frequency]
    u_WKB.append(u_WKB_temp)
    v_WKB.append(v_WKB_temp)

In [ ]:
# plot PSD frequency band power versus depth

fig,(ax0,ax1) = plt.subplots(2,1,figsize=(12,7),sharey=True,sharex=True)
fig.subplots_adjust(hspace = 0.08)

fig.text(0.5, 0.92, f'{f_name} PSD power - {adcp} - {t_stamp}', ha='center', fontsize=12)
fig.text(0.061, 0.5, 'Depth [m]', va='center', rotation='vertical',fontsize=11)
fig.text(0.91, 0.5, 'Integrated band power [(m/s)$^2$]', va='center', rotation='vertical',fontsize=11)
fig.text(0.5, 0.05, f'Months in {t_stamp} [mm]', ha='center',fontsize=11)

if f_name == 'Diurnal':  
    if adcp == 'Axis55' or adcp == 'Axis75':
        vmin = 8e-5
        vmax = 5e-3
        fig.text(0.5, 0.885, 'Cross-canyon', ha='center', fontsize=11)
        fig.text(0.5, 0.485, 'Along-canyon', ha='center', fontsize=11)
    elif adcp == 'Slope':
        vmin = 8e-5
        vmax = 1e-3
        fig.text(0.5, 0.885, 'Cross-slope', ha='center', fontsize=11)
        fig.text(0.5, 0.485, 'Along-slope', ha='center', fontsize=11)
elif f_name == 'Inertial':  
    if adcp == 'Axis55' or adcp == 'Axis75':
        vmin = 8e-5
        vmax = 5e-3
        fig.text(0.5, 0.885, 'Cross-canyon', ha='center', fontsize=11)
        fig.text(0.5, 0.485, 'Along-canyon', ha='center', fontsize=11)
    elif adcp == 'Slope':
        vmin = 1e-4
        vmax = 1e-2
        fig.text(0.5, 0.885, 'Cross-slope', ha='center', fontsize=11)
        fig.text(0.5, 0.485, 'Along-slope', ha='center', fontsize=11)

for i in range(n_seg):
    im0 = ax0.pcolormesh(ds[i].spect_s, -depth, u_WKB[i], rasterized=True, \
                            norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  
    im1 = ax1.pcolormesh(ds[i].spect_s, -depth, v_WKB[i], rasterized=True, \
                            norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  

cbar0 = fig.colorbar(im0, ax=ax0, fraction=0.05, pad=0.01, aspect=40, extend='both')
cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.01, aspect=40, extend='both')  

ax0.patch.set_facecolor('grey')
ax1.patch.set_facecolor('grey')

date_form = pldates.DateFormatter("%m")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_xlim(start_date,end_date)

plt.show()

plt.savefig(fname=f'./plots/depthband_plots/{f_name}_psd_{adcp}_{t_stamp}.pdf',format='pdf')

### Plot w/ barotropic

In [25]:
# plot PSD frequency band power versus depth w/ barotropic time series

# ds_vel_in = xr.open_dataset(f'../Data/data/barotropic/{f_name}_bp_{adcp}_{year}_0.nc')
# n_seg_vel = ds_vel_in.n_seg
# if n_seg_vel > 1:
#     ds_vel = [ds_vel_in]
#     for i in range(n_seg_vel):
#         if i > 0:
#             ds_temp_vel = xr.open_dataset(f'../Data/data/barotropic/{f_name}_bp_{adcp}_{year}_{i}.nc')
#             ds_vel.append(ds_temp_vel)
# elif n_seg_vel == 1:
#     ds_vel = [ds_vel_in]

# fig,(ax0,ax1) = plt.subplots(2,1,figsize=(12,7),sharey=True,sharex=True)
# fig.subplots_adjust(hspace = 0.08)

# fig.text(0.5, 0.92, f'{f_name} PSD power w/ barotropic - {adcp} - {t_stamp}', ha='center', fontsize=12)
# fig.text(0.061, 0.5, 'Depth [m]', va='center', rotation='vertical',fontsize=11)
# fig.text(0.91, 0.5, 'Integrated band power [(m/s)$^2$]', va='center', rotation='vertical',fontsize=11)
# fig.text(0.5, 0.05, f'Months in {t_stamp} [mm]', ha='center',fontsize=11)

# if f_name == 'Diurnal':  
#     if adcp == 'Axis55' or adcp == 'Axis75':
#         vmin = 8e-5
#         vmax = 5e-3
#         fig.text(0.5, 0.885, 'Cross-canyon', ha='center', fontsize=11)
#         fig.text(0.5, 0.485, 'Along-canyon', ha='center', fontsize=11)
#     elif adcp == 'Slope':
#         vmin = 8e-5
#         vmax = 1e-3
#         fig.text(0.5, 0.885, 'Cross-slope', ha='center', fontsize=11)
#         fig.text(0.5, 0.485, 'Along-slope', ha='center', fontsize=11)
# elif f_name == 'Inertial':  
#     if adcp == 'Axis55' or adcp == 'Axis75':
#         vmin = 8e-5
#         vmax = 5e-3
#         fig.text(0.5, 0.885, 'Cross-canyon', ha='center', fontsize=11)
#         fig.text(0.5, 0.485, 'Along-canyon', ha='center', fontsize=11)
#     elif adcp == 'Slope':
#         vmin = 1e-4
#         vmax = 1e-2
#         fig.text(0.5, 0.885, 'Cross-slope', ha='center', fontsize=11)
#         fig.text(0.5, 0.485, 'Along-slope', ha='center', fontsize=11)

# dloc = (depth[-1]+depth[0])/2
# for i in range(n_seg):
#     im0 = ax0.pcolormesh(ds[i].spect_s, -depth, u_WKB[i], rasterized=True, \
#                             norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot') 
#     im0vel = ax0.plot(ds_vel[i].t_seg,(ds_vel[i].ubp.mean(dim='depth')*1000)-dloc,color='C0',alpha=0.5)
#     im1 = ax1.pcolormesh(ds[i].spect_s, -depth, v_WKB[i], rasterized=True, \
#                             norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  
#     im1vel = ax1.plot(ds_vel[i].t_seg,(ds_vel[i].vbp.mean(dim='depth')*1000)-dloc,color='C0',alpha=0.5)

# cbar0 = fig.colorbar(im0, ax=ax0, fraction=0.05, pad=0.01, aspect=40, extend='both')
# cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.01, aspect=40, extend='both')  

# ax0.patch.set_facecolor('grey')
# ax1.patch.set_facecolor('grey')

# date_form = pldates.DateFormatter("%m")
# ax1.xaxis.set_major_formatter(date_form)
# ax1.set_xlim(start_date,end_date)

# plt.show()

## Depth-band rotary spectra

In [19]:
# integrate over band for each time step

cw_power, ccw_power = [],[]

for i in range(n_seg):
    t = len(ds[i].spect_s)
    cw_depth_power_temp,ccw_depth_power_temp = [],[]
    for j in range(d):
        cw_time_power_temp,ccw_time_power_temp = [],[]
        for k in range(t):
            cw_PSD_temp = ds[i].cw_d[j,low_f_idx:up_f_idx+1,k]
            cw_power_temp = np.trapz(y=cw_PSD_temp,x=f_band)
            cw_time_power_temp.append(cw_power_temp)
            ccw_PSD_temp = ds[i].ccw_d[j,low_f_idx:up_f_idx+1,k]
            ccw_power_temp = np.trapz(y=ccw_PSD_temp,x=f_band)
            ccw_time_power_temp.append(ccw_power_temp)
        cw_depth_power_temp.append(cw_time_power_temp)
        ccw_depth_power_temp.append(ccw_time_power_temp)
    cw_np = np.asarray(cw_depth_power_temp)
    ccw_np = np.asarray(ccw_depth_power_temp)
    cw_power.append(cw_np)            # n_seg, [depth, time]
    ccw_power.append(ccw_np)            # n_seg, [depth, time]

In [20]:
# WKB scale at each depth

cw_WKB,ccw_WKB = [],[]
for i in range(n_seg):
    cw_WKB_temp,ccw_WKB_temp = [],[]
    for j in range(d):
        cw_WKB_temp.append(cw_power[i][j] / ds[0].WKB_scale[j].values)          # list[depth][frequency]
        ccw_WKB_temp.append(ccw_power[i][j] / ds[0].WKB_scale[j].values)          # list[depth][frequency]
    cw_WKB.append(cw_WKB_temp)
    ccw_WKB.append(ccw_WKB_temp)

In [ ]:
# plot rotary frequency band power versus depth

fig,(ax0,ax1) = plt.subplots(2,1,figsize=(12,7),sharey=True,sharex=True)
fig.subplots_adjust(hspace = 0.08)

fig.text(0.5, 0.92, f'{f_name} rotary power - {adcp} - {t_stamp}', ha='center', fontsize=12)
fig.text(0.061, 0.5, 'Depth [m]', va='center', rotation='vertical',fontsize=11)
fig.text(0.91, 0.5, 'Integrated band power [(m/s)$^2$]', va='center', rotation='vertical',fontsize=11)
fig.text(0.5, 0.05, f'Months in {t_stamp} [mm]', ha='center',fontsize=11)
fig.text(0.5, 0.885, 'CW', ha='center', fontsize=11)
fig.text(0.5, 0.485, 'CCW', ha='center', fontsize=11)  

if f_name == 'Diurnal':  
    if adcp == 'Axis55' or adcp == 'Axis75':
        vmin = 8e-5
        vmax = 5e-3
    elif adcp == 'Slope':
        vmin = 8e-5
        vmax = 1e-3    
elif f_name == 'Inertial':  
    if adcp == 'Axis55' or adcp == 'Axis75':
        vmin = 8e-5
        vmax = 5e-3
    elif adcp == 'Slope':
        vmin = 1e-4
        vmax = 1e-2   

for i in range(n_seg):
    im0 = ax0.pcolormesh(ds[i].spect_s, -depth, cw_WKB[i], rasterized=True, \
                            norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  
    im1 = ax1.pcolormesh(ds[i].spect_s, -depth, ccw_WKB[i], rasterized=True, \
                            norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  

cbar0 = fig.colorbar(im0, ax=ax0, fraction=0.05, pad=0.01, aspect=40, extend='both')
cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.01, aspect=40, extend='both')  

ax0.patch.set_facecolor('grey')
ax1.patch.set_facecolor('grey')

date_form = pldates.DateFormatter("%m")
ax1.xaxis.set_major_formatter(date_form)
ax1.set_xlim(start_date,end_date)

plt.show()

plt.savefig(fname=f'./plots/depthband_plots/{f_name}_rot_{adcp}_{t_stamp}.pdf',format='pdf')

### Plot w/ barotropic

In [26]:
# plot rotary frequency band power versus depth w/ barotropic

# get barotropic data

# ds_vel_in = xr.open_dataset(f'../Data/data/barotropic/{f_name}_bp_{adcp}_{year}_0.nc')
# n_seg_vel = ds_vel_in.n_seg
# if n_seg_vel > 1:
#     ds_vel = [ds_vel_in]
#     for i in range(n_seg_vel):
#         if i > 0:
#             ds_temp_vel = xr.open_dataset(f'../Data/data/barotropic/{f_name}_bp_{adcp}_{year}_{i}.nc')
#             ds_vel.append(ds_temp_vel)
# elif n_seg_vel == 1:
#     ds_vel = [ds_vel_in]

# fig,(ax0,ax1) = plt.subplots(2,1,figsize=(12,7),sharey=True,sharex=True)
# fig.subplots_adjust(hspace = 0.08)

# fig.text(0.5, 0.92, f'{f_name} rotary power w/ barotropic - {adcp} - {t_stamp}', ha='center', fontsize=12)
# fig.text(0.5, 0.05, f'Months in {t_stamp} [mm]', ha='center',fontsize=11)
# fig.text(0.5, 0.885, 'CW', ha='center', fontsize=11)
# fig.text(0.5, 0.485, 'CCW', ha='center', fontsize=11)  

# if f_name == 'Diurnal':  
#     if adcp == 'Axis55' or adcp == 'Axis75':
#         vmin = 8e-5
#         vmax = 5e-3
#     elif adcp == 'Slope':
#         vmin = 8e-5
#         vmax = 1e-3    
# elif f_name == 'Inertial':  
#     if adcp == 'Axis55' or adcp == 'Axis75':
#         vmin = 8e-5
#         vmax = 5e-3
#     elif adcp == 'Slope':
#         vmin = 1e-4
#         vmax = 1e-2   

# dloc = (depth[-1]+depth[0])/2
# for i in range(n_seg):
#     im0 = ax0.pcolormesh(ds[i].spect_s, -depth, cw_WKB[i], rasterized=True, \
#                             norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')
#     im0vel = ax0.plot(ds_vel[i].t_seg,(np.hypot((ds_vel[i].ubp.mean(dim='depth')),(ds_vel[i].vbp.mean(dim='depth')))*1000)-dloc, \
#                             color='C0',alpha=0.5)
#     im1 = ax1.pcolormesh(ds[i].spect_s, -depth, ccw_WKB[i], rasterized=True, \
#                             norm=colors.LogNorm(vmin=vmin, vmax=vmax), cmap='hot')  
#     im1vel = ax1.plot(ds_vel[i].t_seg,(np.hypot((ds_vel[i].ubp.mean(dim='depth')),(ds_vel[i].vbp.mean(dim='depth')))*1000)-dloc, \
#                             color='C0',alpha=0.5)
# cbar0 = fig.colorbar(im0, ax=ax0, fraction=0.05, pad=0.01, aspect=40, extend='both')
# cbar1 = fig.colorbar(im1, ax=ax1, fraction=0.05, pad=0.01, aspect=40, extend='both')  

# ax0.patch.set_facecolor('grey')
# ax1.patch.set_facecolor('grey')

# date_form = pldates.DateFormatter("%m")
# ax1.xaxis.set_major_formatter(date_form)
# ax1.set_xlim(start_date,end_date)

# fig.text(0.061, 0.5, 'Depth [m]', va='center', rotation='vertical',fontsize=11)
# fig.text(0.91, 0.5, 'Integrated band power [(m/s)$^2$]', va='center', rotation='vertical',fontsize=11)

# plt.show()